In [99]:
from duckduckgo_search import DDGS
import re
import os
import pyttsx3

from groq import Groq
if os.getenv("GROQ_API_KEY") is None:
    os.environ["GROQ_API_KEY"] = 

    
from llama_index.llms.ollama import Ollama
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import StorageContext, load_index_from_storage

import os

import whisper
from langdetect import detect
from pytube import YouTube


import tempfile

#_________________________________________________________________________________________________________________

#Youtube Search Video RAG with Vector DB

def startfile(fn):
    os.system('open %s' % fn)

def create_and_open_txt(text, filename):
    # Create and write the text to a txt file
    with open(filename, "w") as file:
        file.write(text)
    startfile(filename)


def delete_audio_file(file_path):
    try:
        os.remove(file_path)
        print(f"Audio file {file_path} deleted successfully.")
    except OSError as e:
        print(f"Error deleting audio file: {e}")

    
def youtube_audio_text(url, path):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    
    output_path = path 
    filename = "audio.mp3"
    audio_stream.download(output_path=output_path, filename=filename)

    print(f"Audio downloaded to {output_path}\{filename}")

    model = whisper.load_model("base")
    result = model.transcribe(path + "\\audio.mp3")
    transcribed_text = result["text"]
    #print(transcribed_text)

    # language = detect(transcribed_text)
    # print(f"Detected language: {language}")
    delete_audio_file(path + "\\audio.mp3")
    
    create_and_open_txt(transcribed_text, path + "\\output_Text.txt")


def Youtube_RAG(model, path):
    llm = Ollama(model="mistral", request_timeout=30.0)

    #inp = input("enter 1 - Hugging Face Embedding \n2 - OpenAI Embedding")
    inp = 1
    if inp == 1:
        #
        Settings.embed_model = HuggingFaceEmbedding(model_name="flax-sentence-embeddings/all_datasets_v4_MiniLM-L6")


    #Settings.llm = Ollama(model="mistral", request_timeout=200.0)
    Settings.llm = Ollama(model=model, request_timeout=200.0)
    
    documents = SimpleDirectoryReader(path).load_data()
    sen_split=TokenTextSplitter()
    pipeline = IngestionPipeline(
        transformations=[sen_split]
    )
    nodes=pipeline.run(show_progress=True,documents=documents, in_place=True)
    index = VectorStoreIndex.from_documents(
        documents, transformations=[sen_split]
    )
    index.storage_context.persist(persist_dir="./indexDB")
    storage_context = StorageContext.from_defaults(persist_dir="./indexDB")
    index = load_index_from_storage(storage_context)
    query_engine = index.as_query_engine()
    response = query_engine.query("Summary of the Document")
    print(response)
    return response
    
#     while True:
#         #
#         user_input = input("Enter 'stop' to end: ")
#         if user_input.lower() == 'stop':
#             print("Stopping the loop.")
#             break
#         else:
#             response = query_engine.query(user_input)
#             print()
#             print(response)
        

def youtube(url):
    path = tempfile.mkdtemp()
    youtube_audio_text(url, path)
    #model_id = input("Enter the model you want to use: 1 - Mistral, 2- Codellama 13B, 3 - llava")
    model_id = 1
    if model_id == 1:
        response = Youtube_RAG("mistral", path)
    elif model_id == 2:
        response = Youtube_RAG("codellama:13b", path)
#     elif model_id == 3:
#         Youtube_RAG("codellama:13b")
    return response

#_________________________________________________________________________________________________________________
    
    
#Not using Groq Max Tokens Completed for Today
def O_LLM_GroQ(query):
    print("Groq!")
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model="mixtral-8x7b-32768",
        #model="gemma-7b-it",
        temperature = 0,
    )

    response = chat_completion.choices[0].message.content
    return response


import google.generativeai as genai

#Gemini 1.0 pro, good not great need to test on 1.5 pro in google hackathon
def O_LLM_gemini(query):
    Gemini_API = 
    model = genai.GenerativeModel('gemini-pro')
    genai.configure(api_key=Gemini_API)
    response = model.generate_content(query)
    resp = response.text
   # print(response.text)
    return resp

def extract_thoughts_actions_output(text):
    thoughts = []
    actions = []
    outputs = []

    thought_pattern = r'Thought \d+:\s*(.*)'
    action_pattern = r'Action \d+:\s*(.*)'
    output_pattern = r'Output \d+:\s*(.*)'

    # Extract Thoughts, Actions, and Output
    thoughts = re.findall(thought_pattern, text)
    actions = re.findall(action_pattern, text)
    outputs = re.findall(output_pattern, text)

    return thoughts, actions, outputs

def extract_info(texts):
    """
    This function extracts tools and inputs from a list of text strings.

    Args:
      texts: A list of strings containing instructions with tools and inputs in brackets.

    Returns:
      A dictionary where keys are tools (e.g., "Search", "Summarize", "Calculate") 
      and values are corresponding inputs (e.g., "funding received by Mistral Ai from investors").
    """
    tools = {}
    for text in texts:
        # Extract tool using regular expression
        tool = re.findall(r'^\w+', text)[0]

        # Extract input using regular expression
        inp = re.findall(r'\[(.*?)\]', text)[0]

        # Add tool and input to the dictionary
        tools[tool] = inp
    return tools



def duck_go(Keyword):
    print(Keyword)
    results = DDGS().text(Keyword, max_results=12)
    bodies = [item['body'] for item in results]
    paragraph = ' '.join(bodies)
    return paragraph

def Calculate(expression):
    print(f"Calculating: {expression}")

    
import subprocess

def execute_python(code):
    #print("Code recieved for execution Terminal: ",code)
    result = subprocess.run(["python", "-c", code], capture_output=True, text=True)
    err = 0
    # Check if there's an error
    if result.returncode != 0:
        print("Error Found")
        err = 1
        return result.stderr, err
    else:
        
        output = result.stdout
        return output, err

#------------------------------------------------------------------------------------------------
def extract_text(input_string, option):
    if option == 1:
        pattern = r'\```python(.*?)\```'
        matches = re.search(pattern, input_string, re.DOTALL)
        if matches:
            return matches.group(1).strip()
        else:
            return None
    else:
        pattern = r'\```(.*?)\```'
        matches = re.search(pattern, input_string, re.DOTALL)
        if matches:
            return matches.group(1).strip()
        else:
            return None
#------------------------------------------------------------------------------------------------
def check_substring(main_string, substring):

    if substring.lower() in main_string.lower():
        return True
    else:
        return False
#------------------------------------------------------------------------------------------------
Error_Counter = 0

def code_processing(answer):
    #answer = O_LLM(query)
    print("Preprocessing code for execution")
    main_string = answer
    substring = "```python"
    substring_sub = "```"
    print("\n\n")
    if check_substring(main_string, substring_sub):
        #print("```, FOUND PREPROCESSING... ")
        
        if check_substring(main_string, substring):
            #print("```python, FOUND PREPROCESSING... ")
            input_string =  answer
            extracted_text = extract_text(input_string, 1)
            
            if extracted_text:
                answer = extracted_text
                #print("Extracted Text: \n", answer)
                code = answer
            else:
                #print("No text found between ``` and ```.")
                code = answer
        else:
            print("")
            if check_substring(main_string, substring_sub):
                print("```python, FOUND PREPROCESSING... ")
                input_string =  answer
                extracted_text = extract_text(input_string, 0)

                if extracted_text:
                    answer = extracted_text
                    #print("Extracted Text: \n", answer)
                    code = answer
                else:
                    print("No text found between ``` and ```.")
                    code = answer
            
    else:
        print("```python ,NOT FOUND")
        code = answer
    print("Code Extracted: ",code)
    code_to_execute = code    
    result, err = execute_python(code_to_execute)
    if err == 0:
        print("Returning Result to Prompt: ", result)
        Error_Counter = 0
        return result
    else:
        Code_error_recur(code_to_execute, result)

        

def Code_error_recur(code_to_execute, result):
    Error_Counter = Error_Counter + 1
    Error_query = "Code: " + code_to_execute + "\n Error: " + result + "\n Dont add any comments and always write code under these tags '```python' and '```' " 
    resp = O_LLM(Error_query)
    code_processing(resp)

    


def calculate(expression):
    try:
        result = eval(expression)
        return result
    except Exception as e:
        return f"Error: {str(e)}"
    
    

def Voice(voice_response):
    text = voice_response
    engine = pyttsx3.init()
    engine.setProperty('rate', 190)    # Speed percent (can go over 100)
    engine.setProperty('volume', 0.9)  # Volume 0-1
    engine.say(text)
    engine.runAndWait()
    
    return "Speaking completed"
    
    
#Afterwards Update match with elif
def handle_request(data, thought, model):
    #
    if "Search" in data:
        output = duck_go(data["Search"])
        param = data["Search"]
#         print("In DuckDuckGo Search Question:",thought)
        print("Duck Duck Go :",output)
        print("\n\n\n")
        prompt = f"consider the text based on the reference, if you found no connection between the Question and Reference, find the most relevant answer for the question \n Question: {thought}\n\n\n Reference: {output}"
        output = Model_selector(model, prompt)
        return output
    elif "Calculate" in data:
        output = Calculate(data["Calculate"])
        return output
    elif "Python" in data:
        output = code_processing(data["Python"])
        return output
    elif "Voice" in data:
        output = Voice(data["Voice"])
        return output
    elif "Youtube" in data:
        output = youtube(data["Youtube"])
        return output
    else:
        print("Invalid key. Please use 'Search' or 'Calculate'")

        
def convert_list_to_dict(data):
    result = {}
    for item in data:
        try:
            key, value = item.split('[', 1)
            value = value.rsplit(']', 1)[0].strip()  # Get text from beginning to last ']'
            if value:  # Check if value is not empty (null)
                result[key.strip()] = value
        except ValueError:
            continue  # Skip to the next iteration if splitting fails
    return result
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------

def extract_thoughts_actions_output(text):
    thoughts = []
    actions = []
    outputs = []

    thought_pattern = r'Thought \d+:\s*(.*)'
    action_pattern = r'Action \d+:\s*([\s\S]*?)(?=(?:Thought \d+|$))'
    output_pattern = r'output:\s*(.*)'

    # Extract Thoughts, Actions, and Output
    thoughts = re.findall(thought_pattern, text, re.IGNORECASE)
    action_matches = re.findall(action_pattern, text, re.IGNORECASE)
    actions = ['\n'.join(action.strip().split('\n')) for action in action_matches]
    outputs = re.findall(output_pattern, text, re.IGNORECASE)

    return thoughts, actions, outputs


def extract_actions(text):
    pattern = r'[Aa]ction(?:[:\s\d]+)?\s*([^\[\]]+\[[^\[\]]+\])'
    matches = re.findall(pattern, text)
    return matches
#------------------------------------------------------------------------------------------------

import re


def Text_to_list(text):
    names = text.split(',')
    return names


def Model_selector(data, prompt):
    if "gemini" in data:
        output = O_LLM_gemini(prompt)
        return output
    elif "mixtral" in data:
        output = O_LLM_GroQ(prompt)
        return output
    elif "openai" in data:
        output = O_LLM_openai(prompt)
        return output
    elif "mistral" in data:
        output = O_LLM_mistral(prompt)
        return output
    else:
        return data

    
def extract_actions(text):
    pattern = r'[Aa]ction(?:[:\s\d]+)?\s*([^\[\]]+\[[^\[\]]+\])'
    matches = re.findall(pattern, text)
    return matches


def contains_action(text):
    pattern = re.compile(r'\b(?:action|Action)\s*:')
    return bool(pattern.search(text))


def between_LLM_and_user(out,inp, model):

    if contains_action(out):
        #

        actions_list = extract_actions(out)
        print("Action Tools using: ",actions_list)
        actions_tools_dic = convert_list_to_dict(actions_list)
        main_dict = actions_tools_dic
        last_key_value = {list(main_dict.keys())[-1]: main_dict[list(main_dict.keys())[-1]]}
        print("Query sending to tools handler",inp)
        ans = handle_request(last_key_value, inp , model)
        return ans
    else:
#         print("Actions not found showing output")
        return out

#------------------------------------------------------------------------------------------------


def Tools_selector(data):
    calculator_tool_prompt = "Calculator[<Your expression to calculate>]"
    search_tool_prompt = "Search[<Key words to search in internet>]"
    python_tool_prompt = "Python[```python <Your python code here> ```]"
    voice_tool_prompt = "Voice[<Your text>]"
    
    if "Calculator" in data:
        output = f"{calculator_tool_prompt}"
        return output
    elif "Internet Search" in data:
        output = f"{search_tool_prompt}"
        return output
    elif "Python Executor" in data:
        output = f"{python_tool_prompt}"
        return output
    elif "Voice" in data:
        output = f"{voice_tool_prompt}"
        return output
#     elif "Youtube" in data:
#         output = f"{}"
#         return output
#     elif "email" in data:
#         output = f"{}"
#         return output
    else:
        return data


def Tools_example_prompt_selector(data):
    example_calculator_tool_prompt = "Action: Calculator[12*34-57]"
    example_search_tool_prompt = "Action: Search[All the influencers of HELIOS Company]"
    example_python_tool_prompt = "Action: Python[```python\na = 12\nb = 10\nc = a+b\nprint(c) ```]"
    example_voice_tool_prompt = "Action: Voice[Hello, I am a Bot, can you hear me]"
    
    if "Calculator" in data:
        output = f"{example_calculator_tool_prompt}"
        return output
    elif "Internet Search" in data:
        output = f"{example_search_tool_prompt}"
        return output
    elif "Python Executor" in data:
        output = f"{example_python_tool_prompt}"
        return output
    elif "Voice" in data:
        output = f"{example_voice_tool_prompt}"
        return output
#     elif "Youtube" in data:
#         output = f"{}"
#         return output
#     elif "email" in data:
#         output = f"{}"
#         return output
    else:
        return data
    
def list_to_string(my_list):
    result_string = ", ".join(my_list) 
    return result_string

def LLM(model, prompt, inp):
    out = Model_selector(model, prompt)
    out = between_LLM_and_user(out, inp, model)
    return out
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------

def Main():
    model  = "gemini"
    tools = "Calculator,Internet Search,Python Executor"
    Rag = "None"

#     prompt = "Hello, who are you"


    tools_list = Text_to_list(tools)
    print("Tools user want to use: ",tools_list)


    prompt_tools_list = []
    prompt_tools_example_list = []
    for i in tools_list:
        print(i)
        tool = Tools_selector(i)
        prompt_tools_list.append(tool)
        tool_example = Tools_example_prompt_selector(i)
        prompt_tools_example_list.append(tool_example)

    if tools_list:
        prompt_tools_list_text = list_to_string(prompt_tools_list)
        prompt_tools_example_text = list_to_string(prompt_tools_example_list)
        
        prompt_part_one = f"""
            Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: \n{prompt_tools_list_text}
            
            Example of Using the tools:
            {prompt_tools_example_text}
            """
        inp = input(">>")
        print("\n\n")
        
        prompt_part_two = f"""
            To use a tool you should write 'Action:' and tool name you want to use, but if user ask any general query you can answer without any tools or actions.
            {prompt_part_one}

            User: {inp}"""
        
        #out = Model_selector(model, prompt_part_two)
        print("Prompt:",prompt_part_two)
        print("LLM_OUTPIT*************")
        out = LLM(model, prompt_part_two, inp)
        print(out)
    else:
        print("none")


if __name__ == "__main__":
  Main()





Tools user want to use:  ['Calculator', 'Internet Search', 'Python Executor']
Calculator
Internet Search
Python Executor
>>which Indian IPL team won today match 



Prompt: 
            To use a tool you should write 'Action:' and tool name you want to use, but if user ask any general query you can answer without any tools or actions.
            
            Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: 
Calculator[<Your expression to calculate>], Search[<Key words to search in internet>], Python[```python <Your python code here> ```]
            
            Example of Using the tools:
            Action: Calculator[12*34-57], Action: Search[All the influencers of HELIOS Company], Action: Python[```python
a = 12
b = 10
c = a+b
print(c) ```]
            

            User: which Indian IPL team won today match 
LLM_OUTPIT*************
Action Tools using:  ['Search[which Indian IPL team won to

In [72]:
mod = values[1]
tools_gsheet = values[2]
rag_gsheet = values[3]

In [73]:
mod

'GEMINI 1.0 PRO'

In [74]:
tools_g

'Calculator, Internet Search, Python Executor'

In [75]:
rag_gsheet

'No'

In [ ]:
def LLM(model, prompt, inp):
    out = Model_selector(model, prompt)
    out = between_LLM_and_user(out, inp, model)
    return out

In [ ]:

#Hi, what is the name of the Claus opus creator? i just want to know
inp = input("Question:")
prompt_part_two = f"""
To use a tool you should write Action: and tool you want to use
{prompt_part_one}
User: Hello how are you?
Bot: I am doing well, thank you for asking. How can I assist you today?
User: {inp}
"""

print(prompt_part_two)

out = Model_selector(model, prompt_part_two)
print(out)
out = between_LLM_and_user(out, inp, model)
print(out)

In [ ]:
import re

def Text_to_list(text):
    names = text.split(',')
    return names

def Model_selector(data, prompt):
    if "gemini" in data:
        output = O_LLM_gemini(prompt)
        return output
    elif "mixtral" in data:
        output = O_LLM_mixtral(prompt)
        return output
    elif "openai" in data:
        output = O_LLM_openai(prompt)
        return output
    elif "mistral" in data:
        output = O_LLM_mistral(prompt)
        return output
    else:
        return data

    
def extract_actions(text):
    pattern = r'[Aa]ction(?:[:\s\d]+)?\s*([^\[\]]+\[[^\[\]]+\])'
    matches = re.findall(pattern, text)
    return matches


def contains_action(text):
    pattern = re.compile(r'\b(?:action|Action)\s*:')
    return bool(pattern.search(text))


def between_LLM_and_user(out,inp, model):
    if contains_action(out):
        actions_list = extract_actions(out)
        print("Action Tools using: ",actions_list)
        actions_tools_dic = convert_list_to_dict(actions_list)
        main_dict = actions_tools_dic
        last_key_value = {list(main_dict.keys())[-1]: main_dict[list(main_dict.keys())[-1]]}
        print("Query sending to tools handler",inp)
        ans = handle_request(last_key_value, inp , model)
        return ans
    else:
#         print("Actions not found showing output")
        return out


model  = "gemini"
tools = "Calculator, Internet Search, Python Executor"
Rag = "None"

prompt = "Hello, who are you"
out = Model_selector(model, prompt)
print(out)

tools_list = Text_to_list(tools)
print("Tools user want to use: ",tools_list)

calculator_tool_prompt = "Calculator[<Your expression to calculate>]"
search_tool_prompt = "Search[<Key words to search in internet>]"
python_tool_prompt = "Python[```python <Your python code here> ```]"
voice_tool_prompt = "Voice[<Your text>]"




In [ ]:
prompt_part_one = f"""
Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: \n{tools_user_selected}
Example of Actions:
{tools_user_selected_example}
"""

In [ ]:
prompt_part_two = f"""
To use a tool you should write Action: and tool you want to use
{prompt_part_one}
User: Hello how are you?
Bot: I am doing well, thank you for asking. How can I assist you today?
User: {inp}
"""

In [4]:
def list_to_string(my_list):
    result_string = ", ".join(my_list) 
    return result_string

In [5]:
prompt_tools_list_text = list_to_string(prompt_tools_list)

In [6]:
prompt_tools_list_text

'Calculator[<Your expression to calculate>], Search[<Key words to search in internet>], Python[```python <Your python code here> ```], Voice[<Your text>]'

In [14]:
RAG
Phone call
Tools: calculator, Internet_Search, Voice, Youtube, pythoncode, email
Embedding model
LLM Model
Chatbot
Prompt Template
Knowledge graphs

SyntaxError: invalid syntax (3652005817.py, line 2)

Hello, I am Gemini, a large multi-modal model, trained by Google. I am designed to provide information and answer questions to the best of my abilities.
Tools user want to use:  ['Calculator', ' Internet Search', ' Python Executor']


In [ ]:
tools_user_selected = f"{calculator_tool_prompt}, {search_tool_prompt}, {python_tool_prompt}, {voice_tool_prompt}"

In [ ]:
example_calculator_tool_prompt = "Action: Calculator[12*34-57]"
example_search_tool_prompt = "Action: Search[All the influencers of HELIOS Company]"
example_python_tool_prompt = "Action: Python[```python\na = 12\nb = 10\nc = a+b\nprint(c) ```]"
example_voice_tool_prompt = "Action: Voice[Hello, I am a Bot, can you hear me]"

tools_user_selected_example = f"{example_calculator_tool_prompt}\n{example_search_tool_prompt}\n{example_python_tool_prompt}\n"

In [ ]:
prompt_part_one = f"""
Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: \n{tools_user_selected}
Example of Actions:
{tools_user_selected_example}
"""
print(prompt_part_one)

In [ ]:

#Hi, what is the name of the Claus opus creator? i just want to know
inp = input("Question:")
prompt_part_two = f"""
To use a tool you should write Action: and tool you want to use
{prompt_part_one}
User: Hello how are you?
Bot: I am doing well, thank you for asking. How can I assist you today?
User: {inp}
"""

print(prompt_part_two)

out = Model_selector(model, prompt_part_two)
print(out)
out = between_LLM_and_user(out, inp, model)
print(out)

In [155]:
tools_user_selected


'Calculator[<Your expression to calculate>], Search[<Key words to search in internet>], Python[```python <Your python code here> ```], Voice[<Your text>]'

In [157]:
prompt_part_one = f"""
Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: \n{tools_user_selected}
Example of Actions:
{tools_user_selected_example}
"""
print(prompt_part_one)


Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: 
Calculator[<Your expression to calculate>], Search[<Key words to search in internet>], Python[```python <Your python code here> ```], Voice[<Your text>]
Example of Actions:
Action: Calculator[12*34-57]
Action: Search[All the influencers of HELIOS Company]
Action: Python[```python
a = 12
b = 10
c = a+b
print(c) ```]




In [161]:
import re

def extract_actions(text):
    pattern = r'[Aa]ction(?:[:\s\d]+)?\s*([^\[\]]+\[[^\[\]]+\])'
    matches = re.findall(pattern, text)
    return matches

def contains_action(text):
    pattern = re.compile(r'\b(?:action|Action)\s*:')
    return bool(pattern.search(text))

def between_LLM_and_user(out,inp, model):
    if contains_action(out):
        actions_list = extract_actions(out)
        print("Action Tools using: ",actions_list)
        actions_tools_dic = convert_list_to_dict(actions_list)
        main_dict = actions_tools_dic
        last_key_value = {list(main_dict.keys())[-1]: main_dict[list(main_dict.keys())[-1]]}
        print("Query sending to tools handler",inp)
        ans = handle_request(last_key_value, inp , model)
        return ans
    else:
#         print("Actions not found showing output")
        return out

#Hi, what is the name of the Claus opus creator? i just want to know
inp = input("Question:")
prompt_part_two = f"""
To use a tool you should write Action: and tool you want to use
{prompt_part_one}
User: Hello how are you?
Bot: I am doing well, thank you for asking. How can I assist you today?
User: {inp}
"""

print(prompt_part_two)

out = Model_selector(model, prompt_part_two)
print(out)
out = between_LLM_and_user(out, inp, model)
print(out)

In [ ]:
out = between_LLM_and_user(out)

In [162]:
#Hi, what is the name of the Claus opus creator? i just want to know
inp = input("Question:")
prompt_part_two = f"""
To use a tool you should write Action: and tool you want to use
{prompt_part_one}
User: Hello how are you?
Bot: I am doing well, thank you for asking. How can I assist you today?
User: {inp}
"""



print(prompt_part_two)

out = Model_selector(model, prompt_part_two)
# print(out)
out = between_LLM_and_user(out)
print(out)



Question:
  @media print {
    .ms-editor-squiggles-container {
      display:none !important;
    }
  }
  .ms-editor-squiggles-container {
    all: initial;
  }when was chatgpt 4 released

To use a tool you should write Action: and tool you want to use

Here are the tools you can use to calculate or collect information from the internet or python code to execute or voice reply, If you need: 
Calculator[<Your expression to calculate>], Search[<Key words to search in internet>], Python[```python <Your python code here> ```], Voice[<Your text>]
Example of Actions:
Action: Calculator[12*34-57]
Action: Search[All the influencers of HELIOS Company]
Action: Python[```python
a = 12
b = 10
c = a+b
print(c) ```]


User: Hello how are you?
Bot: I am doing well, thank you for asking. How can I assist you today?
User: when was chatgpt 4 released

Action: Search[When was chatgpt 4 released]
Action Tools using:  ['Search[When was chatgpt 4 released]']
Query sending to tools handler when was chatgpt 

In [ ]:
def extract_actions(text):
    actions = []

    action_pattern = r'Action \d+:\s*(.*)'

    actions = re.findall(action_pattern, text)

    return thoughts, actions, outputs

In [132]:
def extract_actions(text):
    print("text:", text)
    print("")
    pattern = r'[Aa]ction(?:[:\s\d]+)?\s*([^\[\]]+\[[^\[\]]+\])'
    matches = re.findall(pattern, text)
    return matches


resp = """Action: Python[```python
import pandas as pd

# Create a sample dataset
data = {'Product': ['iPhone', 'iPad', 'MacBook', 'Apple Watch'],
        'Sales': [100, 200, 300, 400]}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Print the DataFrame
print(df) ```]"""

actions_list = extract_actions(out)
print("Action Tools using: ",actions_list)
actions_tools_dic = convert_list_to_dict(actions_list)
main_dict = actions_tools_dic
last_key_value = {list(main_dict.keys())[-1]: main_dict[list(main_dict.keys())[-1]]}
ans = handle_request(last_key_value, "Hi, what is the name of the Claus opus creator? i just want to know")

oice[Hello how are you?
alculator[12*34-57
earch[All the influencers of HELIOS Company
ython[```pytho






In [ ]:

actions_list = extract_action(out)
actions_tools_dic = convert_list_to_dict(actions_list)
print("Action Tools Found: (List) ",actions_tools_dic)

main_dict = actions_tools_dic
last_key_value = {list(main_dict.keys())[-1]: main_dict[list(main_dict.keys())[-1]]}

print("Last Key value: (Dictionary) ",last_key_value)
result = handle_request(last_key_value, i)
            
ans = handle_request(out, "Hi, what is the name of the Claus opus creator? i just want to know")





In [ ]:
ans